In [24]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load datasets
matches_df = pd.read_csv('/content/Punjab/matches.csv')
balls_df = pd.read_csv('/content/Punjab/balls.csv')

# Data preprocessing
matches_df['date'] = pd.to_datetime(matches_df['date'], format='%d-%m-%Y')
kxip_matches = matches_df[
    (matches_df['team1'] == 'Kings XI Punjab') |
    (matches_df['team2'] == 'Kings XI Punjab')
]

# Create match result column for KXIP
kxip_matches['kxip_result'] = np.where(
    kxip_matches['winner'] == 'Kings XI Punjab',
    'Won', 'Lost'
)

# Add match phase columns to balls data
balls_df['phase'] = pd.cut(
    balls_df['over'],
    bins=[-1, 5, 15, 20],
    labels=['Powerplay', 'Middle', 'Death']
)

# Create combined match-ball data for KXIP
kxip_balls = balls_df.merge(
    kxip_matches[['match_id', 'date', 'kxip_result']],
    on='match_id'
)

# Configure Altair settings
alt.data_transformers.disable_max_rows()
KXIP_COLOR = '#C41E3A'  # KXIP team color for visualizations

print(f"Total KXIP matches: {len(kxip_matches)}")
print(f"Total balls data points: {len(kxip_balls)}")

Total KXIP matches: 17
Total balls data points: 4118


In [56]:
# Create pandas DataFrame with auction data
auction_data = {
    'sr_no': range(1, 22),
    'player': [
        'Mitchell Johnson', 'Glenn Maxwell', 'George Bailey', 'Virender Sehwag',
        'Rishi Dhawan', 'Wriddhiman Saha', 'Shaun Marsh', 'Cheteshwar Pujara',
        'Beuran Hendricks', 'Lakshmipathy Balaji', 'Thisara Perera', 'Gurkirat Singh Mann',
        'Murali Kartik', 'Sandeep Sharma', 'Mandeep Hardev Singh', 'Akshar Rajesh Patel',
        'Parvinder Awana', 'Shardul Narendra Thakur', 'Anureet Singh', 'Shivam Sharma',
        'Karanveer Singh'
    ],
    'type': [
        'All-Rounder', 'All-Rounder', 'Batsman', 'Batsman',
        'All-Rounder', 'Wicket Keeper', 'Batsman', 'Batsman',
        'Bowler', 'Bowler', 'All-Rounder', 'Batsman',
        'Bowler', 'Bowler', 'All-Rounder', 'All-Rounder',
        'Bowler', 'Bowler', 'Bowler', 'Bowler',
        'All-Rounder'
    ],
    'price_inr': [
        65000000, 60000000, 32500000, 32000000,
        30000000, 22000000, 22000000, 19000000,
        18000000, 18000000, 16000000, 13000000,
        10000000, 8500000, 8000000, 7500000,
        6500000, 2000000, 2000000, 1000000,
        1000000
    ]
}

import pandas as pd

# Convert to DataFrame
auction_df = pd.DataFrame(auction_data)

# Add formatted price column (in Crores)
auction_df['price_cr'] = (auction_df['price_inr'] / 10000000).round(2)

# Save to CSV
auction_df.to_csv('kxip_auction_2014.csv', index=False)

# Display first few rows and summary
print("KXIP 2014 Auction Data Summary:")
print("\nPlayer Distribution by Type:")
print(auction_df['type'].value_counts())
print("\nTotal Auction Spend:", f"₹{auction_df['price_inr'].sum():,}")
print("\nSample of the data:")
print(auction_df.head().to_string())

KXIP 2014 Auction Data Summary:

Player Distribution by Type:
type
Bowler           8
All-Rounder      7
Batsman          5
Wicket Keeper    1
Name: count, dtype: int64

Total Auction Spend: ₹394,000,000

Sample of the data:
   sr_no            player         type  price_inr  price_cr
0      1  Mitchell Johnson  All-Rounder   65000000      6.50
1      2     Glenn Maxwell  All-Rounder   60000000      6.00
2      3     George Bailey      Batsman   32500000      3.25
3      4   Virender Sehwag      Batsman   32000000      3.20
4      5      Rishi Dhawan  All-Rounder   30000000      3.00


In [61]:
# Update the name mapping dictionary
name_mapping = {
    'GJ Maxwell': 'Glenn Maxwell',
    'GJ Bailey': 'George Bailey',
    'V Sehwag': 'Virender Sehwag',
    'R Dhawan': 'Rishi Dhawan',
    'WP Saha': 'Wriddhiman Saha',
    'SE Marsh': 'Shaun Marsh',
    'CA Pujara': 'Cheteshwar Pujara',
    'BE Hendricks': 'Beuran Hendricks',
    'L Balaji': 'Lakshmipathy Balaji',
    'NLTC Perera': 'Thisara Perera',
    'M Kartik': 'Murali Kartik',
    'Sandeep Sharma': 'Sandeep Sharma',
    'AR Patel': 'Akshar Rajesh Patel',
    'P Awana': 'Parvinder Awana',
    'Shivam Sharma': 'Shivam Sharma',
    'Karanveer Singh': 'Karanveer Singh',
    'MG Johnson': 'Mitchell Johnson',
    'Mandeep Singh': 'Mandeep Hardev Singh'  # Updated mapping
}

# Get all unique players from ball-by-ball data
batting_players = set(balls_df[balls_df['batting_team'] == 'Kings XI Punjab']['batsman'].unique())
bowling_players = set(balls_df[balls_df['bowling_team'] == 'Kings XI Punjab']['bowler'].unique())
all_match_players = batting_players.union(bowling_players)

# Get auction players
auction_players = set(auction_df['player'].unique())

# Verification Report
print("KXIP 2014 Player Verification Report")
print("=" * 60)

print("\n1. Players in Match Data:")
print("-" * 60)
for player in sorted(all_match_players):
    mapped_name = name_mapping.get(player, player)
    in_auction = mapped_name in auction_players
    print(f"{player:20} → {mapped_name:25} {'(In auction)' if in_auction else '(Retained)'}")

print("\n2. Auction Players Not Found in Matches:")
print("-" * 60)
for player in sorted(auction_players):
    reverse_mapped = False
    for match_name, auction_name in name_mapping.items():
        if auction_name == player and match_name in all_match_players:
            reverse_mapped = True
            break
    if not reverse_mapped:
        print(f"{player:30} (Price: ₹{auction_df[auction_df['player'] == player]['price_cr'].values[0]} Cr)")

print("\n3. Summary:")
print("-" * 60)
print(f"Total players in matches: {len(all_match_players)}")
print(f"Total players in auction: {len(auction_players)}")
print(f"Players mapped successfully: {sum(1 for p in all_match_players if name_mapping.get(p, p) in auction_players)}")

KXIP 2014 Player Verification Report

1. Players in Match Data:
------------------------------------------------------------
AR Patel             → Akshar Rajesh Patel       (In auction)
BE Hendricks         → Beuran Hendricks          (In auction)
CA Pujara            → Cheteshwar Pujara         (In auction)
DA Miller            → DA Miller                 (Retained)
GJ Bailey            → George Bailey             (In auction)
GJ Maxwell           → Glenn Maxwell             (In auction)
Karanveer Singh      → Karanveer Singh           (In auction)
L Balaji             → Lakshmipathy Balaji       (In auction)
M Kartik             → Murali Kartik             (In auction)
M Vohra              → M Vohra                   (Retained)
MG Johnson           → Mitchell Johnson          (In auction)
Mandeep Singh        → Mandeep Hardev Singh      (In auction)
P Awana              → Parvinder Awana           (In auction)
R Dhawan             → Rishi Dhawan              (In auction)
SE Marsh   

In [58]:
# Add overseas status to the auction data
auction_df['is_overseas'] = auction_df['player'].isin([
    'Mitchell Johnson', 'Glenn Maxwell', 'George Bailey',
    'Shaun Marsh', 'Beuran Hendricks', 'Thisara Perera'
])

# Calculate spending statistics
def analyze_auction_spending():
    # Overall spending analysis
    overseas_stats = auction_df.groupby('is_overseas').agg({
        'player': 'count',
        'price_inr': ['sum', 'mean']
    }).round(2)

    overseas_stats.columns = ['player_count', 'total_spend', 'avg_spend']

    # Analysis by player type
    type_stats = auction_df.groupby(['type', 'is_overseas']).agg({
        'player': 'count',
        'price_inr': ['sum', 'mean']
    }).round(2)

    type_stats.columns = ['player_count', 'total_spend', 'avg_spend']

    return overseas_stats, type_stats

def create_spending_charts(auction_df):
    # Spending distribution chart
    spending_chart = alt.Chart(auction_df).mark_bar().encode(
        x=alt.X('type:N', title='Player Type'),
        y=alt.Y('price_cr:Q', title='Price (Crores)'),
        color=alt.Color('is_overseas:N',
            scale=alt.Scale(domain=[True, False],
                          range=['#DD1F2D', '#808080'])),
        tooltip=['player', 'type', 'price_cr']
    ).properties(
        width=400,
        height=300,
        title='Auction Spending by Player Type'
    )

    return spending_chart

# Run analysis
overseas_stats, type_stats = analyze_auction_spending()

# Print statistics
print("KXIP 2014 Auction Analysis - Overseas vs Indian Players")
print("\nOverall Summary:")
print(f"Total Players: {len(auction_df)}")
print(f"Overseas Players: {len(auction_df[auction_df['is_overseas']])}")
print(f"Indian Players: {len(auction_df[~auction_df['is_overseas']])}")

print("\nSpending Summary (in Crores):")
print("Overseas Players:")
print(f"Total Spend: ₹{overseas_stats.loc[True, 'total_spend']/10000000:.2f} Cr")
print(f"Average Spend: ₹{overseas_stats.loc[True, 'avg_spend']/10000000:.2f} Cr")
print("\nIndian Players:")
print(f"Total Spend: ₹{overseas_stats.loc[False, 'total_spend']/10000000:.2f} Cr")
print(f"Average Spend: ₹{overseas_stats.loc[False, 'avg_spend']/10000000:.2f} Cr")

print("\nSpending by Player Type:")
print(type_stats.to_string())

# Create visualization
create_spending_charts(auction_df)

KXIP 2014 Auction Analysis - Overseas vs Indian Players

Overall Summary:
Total Players: 21
Overseas Players: 6
Indian Players: 15

Spending Summary (in Crores):
Overseas Players:
Total Spend: ₹21.35 Cr
Average Spend: ₹3.56 Cr

Indian Players:
Total Spend: ₹18.05 Cr
Average Spend: ₹1.20 Cr

Spending by Player Type:
                           player_count  total_spend    avg_spend
type          is_overseas                                        
All-Rounder   False                   4     46500000  11625000.00
              True                    3    141000000  47000000.00
Batsman       False                   3     64000000  21333333.33
              True                    2     54500000  27250000.00
Bowler        False                   7     48000000   6857142.86
              True                    1     18000000  18000000.00
Wicket Keeper False                   1     22000000  22000000.00


alt.Chart(...)

In [62]:
def analyze_roi():
    # Calculate batting ROI metrics
    batting_roi = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman').agg({
        'runs_batsman': 'sum',
        'ball': 'count',
        'match_id': 'nunique'
    }).reset_index()

    # Add auction prices to batting data
    batting_roi['mapped_name'] = batting_roi['batsman'].map(name_mapping)
    batting_roi = batting_roi.merge(
        auction_df[['player', 'price_cr', 'type']],
        left_on='mapped_name',
        right_on='player',
        how='left'
    )
    batting_roi['runs_per_crore'] = (batting_roi['runs_batsman'] / batting_roi['price_cr']).round(2)
    batting_roi['strike_rate'] = (batting_roi['runs_batsman'] / batting_roi['ball'] * 100).round(2)

    # Calculate bowling ROI metrics
    bowling_roi = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler').agg({
        'total_runs': 'sum',
        'ball': 'count',
        'wicket_type': lambda x: x.notna().sum(),
        'match_id': 'nunique'
    }).reset_index()

    # Add auction prices to bowling data
    bowling_roi['mapped_name'] = bowling_roi['bowler'].map(name_mapping)
    bowling_roi = bowling_roi.merge(
        auction_df[['player', 'price_cr', 'type']],
        left_on='mapped_name',
        right_on='player',
        how='left'
    )
    bowling_roi['wickets_per_crore'] = (bowling_roi['wicket_type'] / bowling_roi['price_cr']).round(2)
    bowling_roi['economy'] = (bowling_roi['total_runs'] / (bowling_roi['ball']/6)).round(2)

    return batting_roi, bowling_roi

def create_roi_charts(batting_roi, bowling_roi):
    # Batting ROI Chart
    batting_chart = alt.Chart(batting_roi[batting_roi['price_cr'].notna()]).mark_circle(size=100).encode(
        x=alt.X('price_cr:Q', title='Auction Price (Crores)'),
        y=alt.Y('runs_batsman:Q', title='Total Runs'),
        size=alt.Size('runs_per_crore:Q', title='Runs per Crore'),
        color=alt.Color('type:N', scale=alt.Scale(scheme='category10')),
        tooltip=['batsman', 'runs_batsman', 'price_cr', 'runs_per_crore', 'strike_rate']
    ).properties(
        width=400,
        height=300,
        title='Batting ROI Analysis'
    )

    # Bowling ROI Chart
    bowling_chart = alt.Chart(bowling_roi[bowling_roi['price_cr'].notna()]).mark_circle(size=100).encode(
        x=alt.X('price_cr:Q', title='Auction Price (Crores)'),
        y=alt.Y('wicket_type:Q', title='Total Wickets'),
        size=alt.Size('wickets_per_crore:Q', title='Wickets per Crore'),
        color=alt.Color('type:N', scale=alt.Scale(scheme='category10')),
        tooltip=['bowler', 'wicket_type', 'price_cr', 'wickets_per_crore', 'economy']
    ).properties(
        width=400,
        height=300,
        title='Bowling ROI Analysis'
    )

    return alt.hconcat(batting_chart, bowling_chart)

# Execute analysis
batting_roi, bowling_roi = analyze_roi()

# Print ROI Analysis
print("Batting ROI Analysis (minimum 100 runs):")
print(batting_roi[
    (batting_roi['price_cr'].notna()) &
    (batting_roi['runs_batsman'] >= 100)
].sort_values('runs_per_crore', ascending=False)[
    ['batsman', 'runs_batsman', 'price_cr', 'runs_per_crore', 'strike_rate']
].to_string(index=False))

print("\nBowling ROI Analysis (minimum 5 wickets):")
print(bowling_roi[
    (bowling_roi['price_cr'].notna()) &
    (bowling_roi['wicket_type'] >= 5)
].sort_values('wickets_per_crore', ascending=False)[
    ['bowler', 'wicket_type', 'price_cr', 'wickets_per_crore', 'economy']
].to_string(index=False))

# Calculate type-wise averages
print("\nValue Analysis by Player Type:")
for player_type in batting_roi['type'].unique():
    type_data = batting_roi[batting_roi['type'] == player_type]
    if not type_data.empty:
        avg_runs_per_crore = type_data['runs_per_crore'].mean()
        print(f"\n{player_type}:")
        print(f"Average Runs per Crore: {avg_runs_per_crore:.2f}")

# Create and display visualization
create_roi_charts(batting_roi, bowling_roi)

Batting ROI Analysis (minimum 100 runs):
   batsman  runs_batsman  price_cr  runs_per_crore  strike_rate
   WP Saha           362      2.20          164.55       141.96
  V Sehwag           455      3.20          142.19       136.23
GJ Maxwell           552      6.00           92.00       176.92
 GJ Bailey           257      3.25           79.08       135.26
 CA Pujara           125      1.90           65.79        96.90

Bowling ROI Analysis (minimum 5 wickets):
         bowler  wicket_type  price_cr  wickets_per_crore  economy
Karanveer Singh           12      0.10             120.00     7.81
       AR Patel           19      0.75              25.33     6.31
 Sandeep Sharma           19      0.85              22.35     8.42
        P Awana            9      0.65              13.85    10.19
       L Balaji           12      1.80               6.67     8.67
       R Dhawan           15      3.00               5.00     7.61
     MG Johnson           17      6.50               2.62     7

alt.HConcatChart(...)

In [67]:
import pandas as pd
import numpy as np
import altair as alt

def analyze_performance_benchmarks():
    # Batting performance metrics
    batting_perf = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman').agg({
        'runs_batsman': ['sum', 'count'],
        'match_id': 'nunique',
        'wicket_type': lambda x: x.notna().sum()
    }).reset_index()

    # Fix column names after aggregation
    batting_perf.columns = ['batsman', 'total_runs', 'balls_faced', 'num_matches', 'dismissals']
    batting_perf['strike_rate'] = (batting_perf['total_runs'] / batting_perf['balls_faced'] * 100).round(2)
    batting_perf['average'] = (batting_perf['total_runs'] / batting_perf['dismissals']).fillna(batting_perf['total_runs']).round(2)

    # Add auction data
    batting_perf['mapped_name'] = batting_perf['batsman'].map(name_mapping)
    batting_benchmarks = batting_perf.merge(
        auction_df[['player', 'price_cr', 'type', 'is_overseas']],
        left_on='mapped_name',
        right_on='player',
        how='left'
    )

    # Bowling performance metrics
    bowling_perf = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler').agg({
        'total_runs': 'sum',
        'ball': 'count',
        'wicket_type': lambda x: x.notna().sum(),
        'match_id': 'nunique'
    }).reset_index()

    bowling_perf.columns = ['bowler', 'runs_conceded', 'balls_bowled', 'wickets', 'num_matches']
    bowling_perf['economy'] = (bowling_perf['runs_conceded'] / (bowling_perf['balls_bowled']/6)).round(2)
    bowling_perf['strike_rate'] = (bowling_perf['balls_bowled'] / bowling_perf['wickets']).fillna(0).round(2)
    bowling_perf['average'] = (bowling_perf['runs_conceded'] / bowling_perf['wickets']).fillna(0).round(2)

    bowling_perf['mapped_name'] = bowling_perf['bowler'].map(name_mapping)
    bowling_benchmarks = bowling_perf.merge(
        auction_df[['player', 'price_cr', 'type', 'is_overseas']],
        left_on='mapped_name',
        right_on='player',
        how='left'
    )

    return batting_benchmarks, bowling_benchmarks

def calculate_performance_indices(batting_benchmarks, bowling_benchmarks):
    # Batting index components
    batting_benchmarks['runs_index'] = (batting_benchmarks['total_runs'] / batting_benchmarks['total_runs'].mean())
    batting_benchmarks['sr_index'] = (batting_benchmarks['strike_rate'] / batting_benchmarks['strike_rate'].mean())
    batting_benchmarks['avg_index'] = (batting_benchmarks['average'] / batting_benchmarks['average'].mean())

    # Overall batting index
    batting_benchmarks['performance_index'] = (
        batting_benchmarks['runs_index'] * 0.4 +
        batting_benchmarks['sr_index'] * 0.3 +
        batting_benchmarks['avg_index'] * 0.3
    ).round(2)

    # Bowling index components
    max_economy = bowling_benchmarks['economy'].max()
    max_average = bowling_benchmarks['average'].replace(0, np.inf).max()

    bowling_benchmarks['wickets_index'] = (bowling_benchmarks['wickets'] / bowling_benchmarks['wickets'].mean())
    bowling_benchmarks['economy_index'] = ((max_economy - bowling_benchmarks['economy']) / max_economy)
    bowling_benchmarks['avg_index'] = ((max_average - bowling_benchmarks['average']) / max_average)

    # Overall bowling index
    bowling_benchmarks['performance_index'] = (
        bowling_benchmarks['wickets_index'] * 0.4 +
        bowling_benchmarks['economy_index'] * 0.3 +
        bowling_benchmarks['avg_index'] * 0.3
    ).round(2)

    return batting_benchmarks, bowling_benchmarks

# Execute analysis
batting_benchmarks, bowling_benchmarks = analyze_performance_benchmarks()
batting_benchmarks, bowling_benchmarks = calculate_performance_indices(batting_benchmarks, bowling_benchmarks)

# Calculate value for money metrics
batting_benchmarks['value_ratio'] = (batting_benchmarks['performance_index'] / batting_benchmarks['price_cr']).round(2)
bowling_benchmarks['value_ratio'] = (bowling_benchmarks['performance_index'] / bowling_benchmarks['price_cr']).round(2)

# Print analysis
print("KXIP 2014 Performance Benchmarking Analysis")
print("\n1. Batting Performance (minimum 100 runs):")
print(batting_benchmarks[batting_benchmarks['total_runs'] >= 100].sort_values('performance_index', ascending=False)[
    ['batsman', 'total_runs', 'strike_rate', 'average', 'price_cr', 'performance_index', 'value_ratio']
].to_string(index=False))

print("\n2. Bowling Performance (minimum 5 wickets):")
print(bowling_benchmarks[bowling_benchmarks['wickets'] >= 5].sort_values('performance_index', ascending=False)[
    ['bowler', 'wickets', 'economy', 'average', 'price_cr', 'performance_index', 'value_ratio']
].to_string(index=False))

print("\n3. Best Value for Money (Batting):")
print(batting_benchmarks[
    (batting_benchmarks['price_cr'].notna()) &
    (batting_benchmarks['total_runs'] >= 100)
].nlargest(5, 'value_ratio')[
    ['batsman', 'total_runs', 'performance_index', 'price_cr', 'value_ratio']
].to_string(index=False))

print("\n4. Best Value for Money (Bowling):")
print(bowling_benchmarks[
    (bowling_benchmarks['price_cr'].notna()) &
    (bowling_benchmarks['wickets'] >= 5)
].nlargest(5, 'value_ratio')[
    ['bowler', 'wickets', 'performance_index', 'price_cr', 'value_ratio']
].to_string(index=False))

KXIP 2014 Performance Benchmarking Analysis

1. Batting Performance (minimum 100 runs):
   batsman  total_runs  strike_rate  average  price_cr  performance_index  value_ratio
GJ Maxwell         552       176.92    32.47      6.00               1.82         0.30
 DA Miller         446       145.28    49.56       NaN               1.48          NaN
  V Sehwag         455       136.23    25.28      3.20               1.47         0.46
   WP Saha         362       141.96    30.17      2.20               1.26         0.57
   M Vohra         324       132.79    46.29       NaN               1.15          NaN
 GJ Bailey         257       135.26    25.70      3.25               0.99         0.30
 CA Pujara         125        96.90    31.25      1.90               0.57         0.30

2. Bowling Performance (minimum 5 wickets):
         bowler  wickets  economy  average  price_cr  performance_index  value_ratio
       AR Patel       19     6.31    22.32      0.75               1.12         1.49
 

In [63]:
def analyze_player_utilization():
    # Calculate matches played for each player
    batting_matches = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman')['match_id'].nunique().reset_index()
    bowling_matches = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler')['match_id'].nunique().reset_index()

    # Combine batting and bowling appearances
    all_matches = pd.concat([
        batting_matches.rename(columns={'batsman': 'player'}),
        bowling_matches.rename(columns={'bowler': 'player'})
    ]).groupby('player')['match_id'].max().reset_index()

    # Add auction data
    all_matches['mapped_name'] = all_matches['player'].map(name_mapping)
    utilization_stats = all_matches.merge(
        auction_df[['player', 'price_cr', 'type', 'is_overseas']],
        left_on='mapped_name',
        right_on='player',
        how='left'
    )

    # Calculate utilization percentage
    total_matches = len(kxip_matches)
    utilization_stats['utilization_pct'] = (utilization_stats['match_id'] / total_matches * 100).round(2)

    return utilization_stats

def create_utilization_charts(utilization_stats):
    # Utilization by price bracket
    price_chart = alt.Chart(utilization_stats[utilization_stats['price_cr'].notna()]).mark_bar().encode(
        x=alt.X('price_cr:Q', title='Auction Price (Crores)'),
        y=alt.Y('utilization_pct:Q', title='Utilization %'),
        color=alt.Color('is_overseas:N',
            scale=alt.Scale(domain=[True, False],
                          range=['#DD1F2D', '#808080'])),
        tooltip=['player_x', 'match_id', 'utilization_pct', 'price_cr', 'type']
    ).properties(
        width=400,
        height=300,
        title='Player Utilization vs Auction Price'
    )

    return price_chart

# Execute analysis
utilization_stats = analyze_player_utilization()

# Print utilization analysis
print("Player Utilization Analysis:")
print("\nOverall Utilization Summary:")
print(f"Total Team Matches: {len(kxip_matches)}")
print(f"Total Players Used: {len(utilization_stats)}")

print("\nUtilization by Price Bracket:")
price_brackets = [0, 1, 3, 7]
for i in range(len(price_brackets)-1):
    bracket_players = utilization_stats[
        (utilization_stats['price_cr'] > price_brackets[i]) &
        (utilization_stats['price_cr'] <= price_brackets[i+1])
    ]
    print(f"\nPrice {price_brackets[i]}-{price_brackets[i+1]} Cr:")
    print(bracket_players[['player_x', 'match_id', 'utilization_pct', 'price_cr', 'type']].sort_values('utilization_pct', ascending=False).to_string(index=False))

print("\nOverseas Player Utilization:")
overseas_stats = utilization_stats[utilization_stats['is_overseas'] == True].sort_values('match_id', ascending=False)
print(overseas_stats[['player_x', 'match_id', 'utilization_pct', 'type']].to_string(index=False))

# Display visualization
create_utilization_charts(utilization_stats)

Player Utilization Analysis:

Overall Utilization Summary:
Total Team Matches: 17
Total Players Used: 19

Utilization by Price Bracket:

Price 0-1 Cr:
       player_x  match_id  utilization_pct  price_cr        type
       AR Patel        17           100.00      0.75 All-Rounder
 Sandeep Sharma        11            64.71      0.85      Bowler
        P Awana         7            41.18      0.65      Bowler
Karanveer Singh         5            29.41      0.10 All-Rounder
  Shivam Sharma         4            23.53      0.10      Bowler
  Mandeep Singh         3            17.65      0.80 All-Rounder
       M Kartik         2            11.76      1.00      Bowler

Price 1-3 Cr:
    player_x  match_id  utilization_pct  price_cr          type
    R Dhawan        13            76.47       3.0   All-Rounder
     WP Saha        13            76.47       2.2 Wicket Keeper
    L Balaji         9            52.94       1.8        Bowler
   CA Pujara         6            35.29       1.9       Ba

alt.Chart(...)

In [68]:
def analyze_role_value():
    # Create performance data for each role
    def calculate_role_stats():
        # Get batting and bowling stats
        batting_stats = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman').agg({
            'runs_batsman': 'sum',
            'ball': 'count',
            'match_id': 'nunique',
            'wicket_type': lambda x: x.notna().sum()
        }).reset_index()

        bowling_stats = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler').agg({
            'total_runs': 'sum',
            'ball': 'count',
            'wicket_type': lambda x: x.notna().sum(),
            'match_id': 'nunique'
        }).reset_index()

        # Add player role information
        batting_stats['mapped_name'] = batting_stats['batsman'].map(name_mapping)
        bowling_stats['mapped_name'] = bowling_stats['bowler'].map(name_mapping)

        # Merge with auction data
        batting_role = batting_stats.merge(
            auction_df[['player', 'type', 'price_cr']],
            left_on='mapped_name',
            right_on='player',
            how='left'
        )

        bowling_role = bowling_stats.merge(
            auction_df[['player', 'type', 'price_cr']],
            left_on='mapped_name',
            right_on='player',
            how='left'
        )

        return batting_role, bowling_role

    batting_role, bowling_role = calculate_role_stats()

    # Analyze performance by role
    role_summary = auction_df.groupby('type').agg({
        'price_cr': ['sum', 'mean', 'count']
    }).round(2)

    role_summary.columns = ['total_investment', 'avg_investment', 'player_count']

    # Calculate role-specific metrics
    def get_role_metrics():
        role_metrics = pd.DataFrame()
        for role in auction_df['type'].unique():
            role_players = auction_df[auction_df['type'] == role]['player'].tolist()
            role_data = {
                'role': role,
                'total_runs': batting_role[batting_role['type'] == role]['runs_batsman'].sum(),
                'total_wickets': bowling_role[bowling_role['type'] == role]['wicket_type'].sum(),
                'matches_played': max(
                    batting_role[batting_role['type'] == role]['match_id'].sum(),
                    bowling_role[bowling_role['type'] == role]['match_id'].sum()
                ),
                'investment': auction_df[auction_df['type'] == role]['price_cr'].sum()
            }
            role_metrics = pd.concat([role_metrics, pd.DataFrame([role_data])], ignore_index=True)

        role_metrics['runs_per_cr'] = (role_metrics['total_runs'] / role_metrics['investment']).round(2)
        role_metrics['wickets_per_cr'] = (role_metrics['total_wickets'] / role_metrics['investment']).round(2)

        return role_metrics

    role_metrics = get_role_metrics()

    return role_summary, role_metrics, batting_role, bowling_role

# Execute analysis
role_summary, role_metrics, batting_role, bowling_role = analyze_role_value()

# Print analysis results
print("KXIP 2014 Role-based Value Assessment")
print("\n1. Investment by Role:")
print(role_summary)

print("\n2. Role Performance Metrics:")
print(role_metrics[['role', 'total_runs', 'total_wickets', 'investment', 'runs_per_cr', 'wickets_per_cr']].to_string(index=False))

# Analyze all-rounders specifically
print("\n3. All-rounder Performance:")
allrounders = pd.merge(
    batting_role[batting_role['type'] == 'All-Rounder'],
    bowling_role[bowling_role['type'] == 'All-Rounder'],
    on='mapped_name',
    suffixes=('_bat', '_bowl')
)

if not allrounders.empty:
    print(allrounders[[
        'mapped_name', 'runs_batsman', 'wicket_type_bowl', 'price_cr_bat'
    ]].sort_values('price_cr_bat', ascending=False).to_string(index=False))

# Create effectiveness visualizations
def create_role_charts():
    # Role investment breakdown
    investment_chart = alt.Chart(role_metrics).mark_bar().encode(
        x=alt.X('role:N', title='Role'),
        y=alt.Y('investment:Q', title='Investment (Crores)'),
        color=alt.Color('role:N', scale=alt.Scale(scheme='category10')),
        tooltip=['role', 'investment', 'total_runs', 'total_wickets']
    ).properties(
        width=300,
        height=200,
        title='Investment by Role'
    )

    # Role effectiveness
    effectiveness_chart = alt.Chart(role_metrics).mark_bar().encode(
        x=alt.X('role:N', title='Role'),
        y=alt.Y('runs_per_cr:Q', title='Runs per Crore'),
        color=alt.Color('role:N', scale=alt.Scale(scheme='category10')),
        tooltip=['role', 'runs_per_cr', 'wickets_per_cr']
    ).properties(
        width=300,
        height=200,
        title='Performance Value by Role'
    )

    return alt.hconcat(investment_chart, effectiveness_chart)

# Display visualizations
create_role_charts()

KXIP 2014 Role-based Value Assessment

1. Investment by Role:
               total_investment  avg_investment  player_count
type                                                         
All-Rounder               18.75            2.68             7
Batsman                   11.85            2.37             5
Bowler                     6.60            0.82             8
Wicket Keeper              2.20            2.20             1

2. Role Performance Metrics:
         role  total_runs  total_wickets  investment  runs_per_cr  wickets_per_cr
  All-Rounder         777             64       18.75        41.44            3.41
      Batsman         907              0       11.85        76.54            0.00
Wicket Keeper         362              0        2.20       164.55            0.00
       Bowler           8             49        6.60         1.21            7.42

3. All-rounder Performance:
        mapped_name  runs_batsman  wicket_type_bowl  price_cr_bat
   Mitchell Johnson            

alt.HConcatChart(...)

In [72]:
def analyze_strategic_investment():
    # Core vs Support Player Analysis
    def identify_core_players():
        # Core players are those who played more than 50% of matches or were high-priced
        total_matches = len(kxip_matches)
        median_price = auction_df['price_cr'].median()

        # Get player appearances
        batting_apps = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman')['match_id'].nunique()
        bowling_apps = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler')['match_id'].nunique()
        appearances = pd.concat([batting_apps, bowling_apps]).groupby(level=0).max()

        # Create player classification DataFrame
        player_class = pd.DataFrame(appearances).reset_index()
        player_class.columns = ['player_name', 'matches_played']
        player_class['mapped_name'] = player_class['player_name'].map(name_mapping)

        # Merge with auction data
        player_class = player_class.merge(
            auction_df[['player', 'price_cr', 'type', 'is_overseas']],
            left_on='mapped_name',
            right_on='player',
            how='left'
        )

        # Classify players
        player_class['is_core'] = (
            (player_class['matches_played'] > total_matches/2) |
            (player_class['price_cr'] > median_price)
        )

        return player_class

    # Investment Distribution Analysis
    def analyze_investment_distribution():
        price_data = auction_df.copy()
        price_data['price_bracket'] = pd.cut(
            price_data['price_cr'],
            bins=[0, 1, 4, 8],
            labels=['Budget (< 1 Cr)', 'Mid-range (1-5 Cr)', 'Premium (> 5 Cr)']
        )

        dist_summary = price_data.groupby('price_bracket').agg({
            'player': 'count',
            'price_cr': 'sum',
            'is_overseas': 'sum'
        }).round(2)

        dist_summary.columns = ['player_count', 'total_investment', 'overseas_count']

        return dist_summary, price_data

    # Calculate strategy metrics
    player_classification = identify_core_players()
    investment_dist, price_data = analyze_investment_distribution()

    return player_classification, investment_dist, price_data

def create_strategy_charts(player_classification, price_data):
    # Core vs Support Investment
    core_chart = alt.Chart(player_classification).mark_bar().encode(
        x=alt.X('is_core:N', title='Player Category'),
        y=alt.Y('sum(price_cr):Q', title='Total Investment (Crores)'),
        color=alt.Color('is_core:N', scale=alt.Scale(
            domain=[True, False],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['is_core', 'sum(price_cr)']
    ).properties(
        width=300,
        height=200,
        title='Investment: Core vs Support Players'
    )

    # Investment Distribution
    dist_chart = alt.Chart(price_data).mark_bar().encode(
        x=alt.X('price_bracket:N', title='Price Bracket'),
        y=alt.Y('count():Q', title='Number of Players'),
        color=alt.Color('is_overseas:N', scale=alt.Scale(
            domain=[True, False],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['price_bracket', 'count()', 'is_overseas']
    ).properties(
        width=300,
        height=200,
        title='Player Distribution by Price Bracket'
    )

    return alt.hconcat(core_chart, dist_chart)

# Execute analysis
player_classification, investment_dist, price_data = analyze_strategic_investment()

# Print analysis results
print("KXIP 2014 Strategic Investment Analysis")
print("\n1. Core vs Support Players:")
core_summary = player_classification.groupby('is_core').agg({
    'player_name': 'count',
    'price_cr': ['sum', 'mean']
}).round(2)
print(core_summary)

print("\n2. Investment Distribution:")
print(investment_dist)

print("\n3. Core Players (>50% matches or above median price):")
core_players = player_classification[player_classification['is_core']].sort_values('price_cr', ascending=False)
print(core_players[['player_name', 'matches_played', 'price_cr', 'type']].to_string(index=False))

print("\n4. Overseas Investment:")
overseas_summary = auction_df.groupby('is_overseas').agg({
    'player': 'count',
    'price_cr': ['sum', 'mean']
}).round(2)
print(overseas_summary)

# Display visualizations
create_strategy_charts(player_classification, price_data)

KXIP 2014 Strategic Investment Analysis

1. Core vs Support Players:
        player_name price_cr      
              count      sum  mean
is_core                           
False             6     2.65  0.53
True             13    33.45  2.79

2. Investment Distribution:
                    player_count  total_investment  overseas_count
price_bracket                                                     
Budget (< 1 Cr)                9              4.65               0
Mid-range (1-5 Cr)            10             22.25               4
Premium (> 5 Cr)               2             12.50               2

3. Core Players (>50% matches or above median price):
   player_name  matches_played  price_cr          type
    MG Johnson              14      6.50   All-Rounder
    GJ Maxwell              16      6.00   All-Rounder
     GJ Bailey              16      3.25       Batsman
      V Sehwag              17      3.20       Batsman
      R Dhawan              13      3.00   All-Rounder
      S

alt.HConcatChart(...)

In [71]:
def analyze_risk_return():
    # High-price player analysis (>6 Crores)
    def analyze_high_price_players():
        HIGH_PRICE_THRESHOLD = 6.0

        # Get high-priced players
        high_price = auction_df[auction_df['price_cr'] >= HIGH_PRICE_THRESHOLD].copy()

        # Get batting performance
        batting_perf = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman').agg({
            'runs_batsman': 'sum',
            'ball': 'count',
            'match_id': 'nunique',
            'wicket_type': lambda x: x.notna().sum()
        }).reset_index()
        batting_perf['mapped_name'] = batting_perf['batsman'].map(name_mapping)
        batting_perf['strike_rate'] = (batting_perf['runs_batsman'] / batting_perf['ball'] * 100).round(2)

        # Get bowling performance
        bowling_perf = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler').agg({
            'wicket_type': lambda x: x.notna().sum(),
            'total_runs': 'sum',
            'ball': 'count',
            'match_id': 'nunique'
        }).reset_index()
        bowling_perf['mapped_name'] = bowling_perf['bowler'].map(name_mapping)
        bowling_perf['economy'] = (bowling_perf['total_runs'] / (bowling_perf['ball']/6)).round(2)

        # Merge performance with high price players
        high_price_stats = high_price.merge(
            batting_perf[['mapped_name', 'runs_batsman', 'strike_rate', 'match_id']],
            left_on='player',
            right_on='mapped_name',
            how='left'
        ).merge(
            bowling_perf[['mapped_name', 'wicket_type', 'economy', 'total_runs']],
            on='mapped_name',
            how='left'
        )

        # Calculate performance metrics
        high_price_stats['runs_per_cr'] = (high_price_stats['runs_batsman'] / high_price_stats['price_cr']).round(2)
        high_price_stats['wickets_per_cr'] = (high_price_stats['wicket_type'] / high_price_stats['price_cr']).round(2)

        return high_price_stats

    # Budget player analysis (Bottom 25%)
    def analyze_budget_players():
        budget_threshold = auction_df['price_cr'].quantile(0.25)

        # Get budget players
        budget = auction_df[auction_df['price_cr'] <= budget_threshold].copy()

        # Get batting performance
        batting_perf = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby('batsman').agg({
            'runs_batsman': 'sum',
            'ball': 'count',
            'match_id': 'nunique',
            'wicket_type': lambda x: x.notna().sum()
        }).reset_index()
        batting_perf['mapped_name'] = batting_perf['batsman'].map(name_mapping)
        batting_perf['strike_rate'] = (batting_perf['runs_batsman'] / batting_perf['ball'] * 100).round(2)

        # Get bowling performance
        bowling_perf = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby('bowler').agg({
            'wicket_type': lambda x: x.notna().sum(),
            'total_runs': 'sum',
            'ball': 'count',
            'match_id': 'nunique'
        }).reset_index()
        bowling_perf['mapped_name'] = bowling_perf['bowler'].map(name_mapping)
        bowling_perf['economy'] = (bowling_perf['total_runs'] / (bowling_perf['ball']/6)).round(2)

        # Merge performance with budget players
        budget_stats = budget.merge(
            batting_perf[['mapped_name', 'runs_batsman', 'strike_rate', 'match_id']],
            left_on='player',
            right_on='mapped_name',
            how='left'
        ).merge(
            bowling_perf[['mapped_name', 'wicket_type', 'economy', 'total_runs']],
            on='mapped_name',
            how='left'
        )

        # Calculate performance metrics
        budget_stats['runs_per_cr'] = (budget_stats['runs_batsman'] / budget_stats['price_cr']).round(2)
        budget_stats['wickets_per_cr'] = (budget_stats['wicket_type'] / budget_stats['price_cr']).round(2)

        return budget_stats, budget_threshold

    high_price_stats = analyze_high_price_players()
    budget_stats, budget_threshold = analyze_budget_players()

    return high_price_stats, budget_stats, budget_threshold

def create_risk_return_charts(high_price_stats, budget_stats):
    # High price performance chart
    high_price_chart = alt.Chart(high_price_stats).mark_circle(size=100).encode(
        x=alt.X('price_cr:Q', title='Investment (Crores)'),
        y=alt.Y('runs_per_cr:Q', title='Runs per Crore'),
        color=alt.Color('type:N', scale=alt.Scale(scheme='category10')),
        size=alt.Size('wickets_per_cr:Q', scale=alt.Scale(domain=[0, 20])),
        tooltip=['player', 'price_cr', 'runs_per_cr', 'wickets_per_cr', 'strike_rate', 'economy']
    ).properties(
        width=300,
        height=200,
        title='Premium Player Performance (>6 Cr)'
    )

    # Budget performance chart
    budget_chart = alt.Chart(budget_stats).mark_circle(size=100).encode(
        x=alt.X('price_cr:Q', title='Investment (Crores)'),
        y=alt.Y('runs_per_cr:Q', title='Runs per Crore'),
        color=alt.Color('type:N', scale=alt.Scale(scheme='category10')),
        size=alt.Size('wickets_per_cr:Q', scale=alt.Scale(domain=[0, 20])),
        tooltip=['player', 'price_cr', 'runs_per_cr', 'wickets_per_cr', 'strike_rate', 'economy']
    ).properties(
        width=300,
        height=200,
        title='Budget Player Performance'
    )

    return alt.hconcat(high_price_chart, budget_chart)

# Execute analysis
high_price_stats, budget_stats, budget_threshold = analyze_risk_return()

# Print analysis results
print("KXIP 2014 Risk-Return Analysis")
print("\n1. Premium Players Performance (>6 Cr):")
print(high_price_stats[
    ['player', 'price_cr', 'runs_batsman', 'strike_rate', 'wicket_type', 'economy', 'runs_per_cr', 'wickets_per_cr']
].sort_values('price_cr', ascending=False).to_string(index=False))

print(f"\n2. Budget Players Performance (<₹{budget_threshold:.2f} Cr):")
print(budget_stats[
    ['player', 'price_cr', 'runs_batsman', 'strike_rate', 'wicket_type', 'economy', 'runs_per_cr', 'wickets_per_cr']
].sort_values('runs_per_cr', ascending=False).to_string(index=False))

# Calculate risk metrics
print("\n3. Risk-Return Summary:")
high_price_roi = (high_price_stats['runs_per_cr'].mean() + high_price_stats['wickets_per_cr'].mean() * 20).round(2)
budget_roi = (budget_stats['runs_per_cr'].mean() + budget_stats['wickets_per_cr'].mean() * 20).round(2)
print(f"Premium Player ROI: {high_price_roi}")
print(f"Budget Player ROI: {budget_roi}")

# Display visualizations
create_risk_return_charts(high_price_stats, budget_stats)

KXIP 2014 Risk-Return Analysis

1. Premium Players Performance (>6 Cr):
          player  price_cr  runs_batsman  strike_rate  wicket_type  economy  runs_per_cr  wickets_per_cr
Mitchell Johnson       6.5            56       109.80           17     7.94         8.62            2.62
   Glenn Maxwell       6.0           552       176.92            1    11.40        92.00            0.17

2. Budget Players Performance (<₹0.75 Cr):
                 player  price_cr  runs_batsman  strike_rate  wicket_type  economy  runs_per_cr  wickets_per_cr
    Akshar Rajesh Patel      0.75          62.0       101.64         19.0     6.31        82.67           25.33
          Shivam Sharma      0.10           5.0       166.67          3.0     7.56        50.00           30.00
        Karanveer Singh      0.10           1.0       100.00         12.0     7.81        10.00          120.00
        Parvinder Awana      0.65           NaN          NaN          NaN      NaN          NaN             NaN
Shardul N

alt.HConcatChart(...)

In [31]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def preprocess_data():
    # Create opponent column
    opponents = []
    for idx, match in kxip_matches.iterrows():
        opponent = match['team2'] if match['team1'] == 'Kings XI Punjab' else match['team1']
        opponents.append(opponent)
    kxip_matches['opponent'] = opponents

    # Add match numbers and win details
    kxip_matches['match_number'] = range(1, len(kxip_matches) + 1)
    kxip_matches['cumulative_wins'] = (kxip_matches['winner'] == 'Kings XI Punjab').cumsum()
    kxip_matches['win_details'] = kxip_matches.apply(
        lambda x: f"{x['win_margin']} {x['win_type']}" if pd.notnull(x['win_type']) else "", axis=1
    )

    return kxip_matches

def create_h2h_stats(data):
    h2h_stats = pd.DataFrame({
        'opponent': data['opponent'].unique()
    })

    h2h_stats['matches'] = h2h_stats['opponent'].apply(
        lambda x: len(data[data['opponent'] == x])
    )
    h2h_stats['wins'] = h2h_stats['opponent'].apply(
        lambda x: len(data[(data['opponent'] == x) &
                          (data['winner'] == 'Kings XI Punjab')])
    )
    h2h_stats['losses'] = h2h_stats['matches'] - h2h_stats['wins']
    h2h_stats['win_rate'] = (h2h_stats['wins'] / h2h_stats['matches'] * 100).round(1)

    return h2h_stats

def create_visualizations(data, h2h_stats):
    # Head to Head Chart
    h2h_chart = alt.Chart(h2h_stats.melt(
        id_vars=['opponent', 'win_rate'],
        value_vars=['wins', 'losses']
    )).mark_bar().encode(
        x=alt.X('opponent:N', title='Opponent', sort='-y'),
        y=alt.Y('value:Q', title='Matches'),
        color=alt.Color('variable:N', scale=alt.Scale(
            domain=['wins', 'losses'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['opponent', 'variable', 'value', 'win_rate']
    ).properties(
        width=300,
        height=200,
        title='Head-to-Head Performance'
    )

    # Progression Chart
    base_line = alt.Chart(data).mark_line(
        color='#DD1F2D'
    ).encode(
        x=alt.X('match_number:Q',
                title='Match Number',
                axis=alt.Axis(tickMinStep=1)),
        y=alt.Y('cumulative_wins:Q',
                title='Cumulative Wins',
                axis=alt.Axis(tickMinStep=1))
    )

    points = alt.Chart(data).mark_circle(size=100).encode(
        x='match_number:Q',
        y='cumulative_wins:Q',
        color=alt.Color('kxip_result:N', scale=alt.Scale(
            domain=['Won', 'Lost'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['date', 'opponent', 'winner', 'win_details']
    )

    progression_chart = (base_line + points).properties(
        width=300,
        height=200,
        title='Win Progression'
    )

    # Win Rate Pie Chart
    win_rate = len(data[data['kxip_result'] == 'Won']) / len(data) * 100
    pie_data = pd.DataFrame({
        'category': ['Wins', 'Losses'],
        'value': [win_rate, 100-win_rate],
        'percentage': [f"{win_rate:.1f}%", f"{(100-win_rate):.1f}%"]
    })

    pie_chart = alt.Chart(pie_data).mark_arc().encode(
        theta=alt.Theta(field='value', type='quantitative'),
        color=alt.Color('category:N', scale=alt.Scale(
            domain=['Wins', 'Losses'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['category', 'percentage']
    ).properties(
        width=200,
        height=200,
        title='Win Rate'
    )

    return h2h_chart, progression_chart, pie_chart

def run_analysis():
    # Process data
    processed_data = preprocess_data()
    h2h_stats = create_h2h_stats(processed_data)

    # Create visualizations
    h2h_chart, progression_chart, pie_chart = create_visualizations(processed_data, h2h_stats)

    # Combine charts horizontally
    combined_chart = alt.hconcat(h2h_chart, progression_chart, pie_chart)

    # Print statistics
    print("Season Summary Statistics:")
    print(f"Total Matches: {len(processed_data)}")
    print(f"Wins: {len(processed_data[processed_data['kxip_result'] == 'Won'])}")
    print(f"Win Rate: {(len(processed_data[processed_data['kxip_result'] == 'Won']) / len(processed_data) * 100):.1f}%")
    print("\nHead-to-Head Statistics:")
    print(h2h_stats.sort_values('wins', ascending=False).to_string(index=False))

    return combined_chart

# Execute analysis and display results
final_visualization = run_analysis()
final_visualization

Season Summary Statistics:
Total Matches: 17
Wins: 12
Win Rate: 70.6%

Head-to-Head Statistics:
                   opponent  matches  wins  losses  win_rate
        Chennai Super Kings        3     3       0     100.0
           Rajasthan Royals        2     2       0     100.0
        Sunrisers Hyderabad        2     2       0     100.0
Royal Challengers Bangalore        2     2       0     100.0
           Delhi Daredevils        2     2       0     100.0
      Kolkata Knight Riders        4     1       3      25.0
             Mumbai Indians        2     0       2       0.0


alt.HConcatChart(...)

In [32]:
import pandas as pd
import numpy as np
import altair as alt

def analyze_venues():
    # Venue performance analysis
    venue_stats = pd.DataFrame({
        'venue': kxip_matches['venue'].unique()
    })

    venue_stats['matches'] = venue_stats['venue'].apply(
        lambda x: len(kxip_matches[kxip_matches['venue'] == x])
    )
    venue_stats['wins'] = venue_stats['venue'].apply(
        lambda x: len(kxip_matches[(kxip_matches['venue'] == x) &
                                 (kxip_matches['winner'] == 'Kings XI Punjab')])
    )
    venue_stats['losses'] = venue_stats['matches'] - venue_stats['wins']
    venue_stats['win_rate'] = (venue_stats['wins'] / venue_stats['matches'] * 100).round(1)

    # Add home/away designation
    venue_stats['location'] = venue_stats['venue'].apply(
        lambda x: 'Home' if 'Punjab' in x or 'Mohali' in x else 'Away'
    )

    return venue_stats

def create_venue_charts(venue_stats):
    # Venue Performance Chart
    venue_chart = alt.Chart(venue_stats.melt(
        id_vars=['venue', 'win_rate', 'location'],
        value_vars=['wins', 'losses']
    )).mark_bar().encode(
        x=alt.X('venue:N', title='Venue', sort='-y'),
        y=alt.Y('value:Q', title='Matches'),
        color=alt.Color('variable:N', scale=alt.Scale(
            domain=['wins', 'losses'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['venue', 'variable', 'value', 'win_rate', 'location']
    ).properties(
        width=400,
        height=200,
        title='Performance by Venue'
    )

    # Home vs Away Summary
    location_summary = venue_stats.groupby('location').agg({
        'matches': 'sum',
        'wins': 'sum',
        'losses': 'sum'
    }).reset_index()
    location_summary['win_rate'] = (location_summary['wins'] / location_summary['matches'] * 100).round(1)

    location_chart = alt.Chart(location_summary.melt(
        id_vars=['location', 'win_rate'],
        value_vars=['wins', 'losses']
    )).mark_bar().encode(
        x=alt.X('location:N', title='Location'),
        y=alt.Y('value:Q', title='Matches'),
        color=alt.Color('variable:N', scale=alt.Scale(
            domain=['wins', 'losses'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['location', 'variable', 'value', 'win_rate']
    ).properties(
        width=200,
        height=200,
        title='Home vs Away Performance'
    )

    # Combine charts horizontally
    combined_chart = alt.hconcat(venue_chart, location_chart)

    return combined_chart, location_summary

# Execute analysis
venue_stats = analyze_venues()
combined_chart, location_summary = create_venue_charts(venue_stats)

# Print statistics
print("Venue Performance Summary:")
print(venue_stats.sort_values('wins', ascending=False).to_string(index=False))
print("\nHome vs Away Summary:")
print(location_summary.to_string(index=False))

# Display visualization
combined_chart

Venue Performance Summary:
                                     venue  matches  wins  losses  win_rate location
                      Sheikh Zayed Stadium        2     2       0     100.0     Away
                   Sharjah Cricket Stadium        2     2       0     100.0     Away
Punjab Cricket Association Stadium, Mohali        3     2       1      66.7     Home
       Dubai International Cricket Stadium        1     1       0     100.0     Away
                          Wankhede Stadium        2     1       1      50.0     Away
                          Barabati Stadium        2     1       1      50.0     Away
                     M Chinnaswamy Stadium        2     1       1      50.0     Away
 Rajiv Gandhi International Stadium, Uppal        1     1       0     100.0     Away
                          Feroz Shah Kotla        1     1       0     100.0     Away
                              Eden Gardens        1     0       1       0.0     Away

Home vs Away Summary:
location  match

alt.HConcatChart(...)

In [33]:
def analyze_toss_innings():
    # Toss analysis
    toss_data = pd.DataFrame({
        'toss_winner': kxip_matches['toss_winner'],
        'toss_decision': kxip_matches['toss_decision'],
        'winner': kxip_matches['winner']
    })

    toss_stats = {
        'toss_wins': len(toss_data[toss_data['toss_winner'] == 'Kings XI Punjab']),
        'bat_first_choice': len(toss_data[(toss_data['toss_winner'] == 'Kings XI Punjab') &
                                        (toss_data['toss_decision'] == 'bat')]),
        'field_first_choice': len(toss_data[(toss_data['toss_winner'] == 'Kings XI Punjab') &
                                          (toss_data['toss_decision'] == 'field')])
    }

    # Create toss decisions chart data
    toss_decision_data = pd.DataFrame({
        'decision': ['Bat', 'Field'],
        'count': [toss_stats['bat_first_choice'], toss_stats['field_first_choice']]
    })

    # Innings analysis
    batting_first = []
    for idx, match in kxip_matches.iterrows():
        if ((match['team1'] == 'Kings XI Punjab' and match['toss_decision'] == 'bat') or
            (match['team2'] == 'Kings XI Punjab' and match['toss_decision'] == 'field')):
            batting_first.append('Yes')
        else:
            batting_first.append('No')

    kxip_matches['batting_first'] = batting_first
    innings_stats = pd.DataFrame({
        'batting_first': ['Yes', 'No'],
        'matches': [
            len(kxip_matches[kxip_matches['batting_first'] == 'Yes']),
            len(kxip_matches[kxip_matches['batting_first'] == 'No'])
        ],
        'wins': [
            len(kxip_matches[(kxip_matches['batting_first'] == 'Yes') &
                            (kxip_matches['winner'] == 'Kings XI Punjab')]),
            len(kxip_matches[(kxip_matches['batting_first'] == 'No') &
                            (kxip_matches['winner'] == 'Kings XI Punjab')])
        ]
    })
    innings_stats['losses'] = innings_stats['matches'] - innings_stats['wins']
    innings_stats['win_rate'] = (innings_stats['wins'] / innings_stats['matches'] * 100).round(1)

    return toss_stats, toss_decision_data, innings_stats

def create_toss_innings_charts(toss_decision_data, innings_stats):
    # Toss decisions chart
    toss_chart = alt.Chart(toss_decision_data).mark_arc().encode(
        theta=alt.Theta(field='count', type='quantitative'),
        color=alt.Color('decision:N', scale=alt.Scale(
            domain=['Bat', 'Field'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['decision', 'count']
    ).properties(
        width=250,
        height=200,
        title='Toss Decisions When Won'
    )

    # Innings performance chart
    innings_chart = alt.Chart(innings_stats.melt(
        id_vars=['batting_first', 'win_rate'],
        value_vars=['wins', 'losses']
    )).mark_bar().encode(
        x=alt.X('batting_first:N', title='Batting First'),
        y=alt.Y('value:Q', title='Matches'),
        color=alt.Color('variable:N', scale=alt.Scale(
            domain=['wins', 'losses'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['batting_first', 'variable', 'value', 'win_rate']
    ).properties(
        width=250,
        height=200,
        title='Performance by Innings'
    )

    # Combine charts horizontally
    combined_chart = alt.hconcat(toss_chart, innings_chart)

    return combined_chart

# Execute analysis
toss_stats, toss_decision_data, innings_stats = analyze_toss_innings()
combined_chart = create_toss_innings_charts(toss_decision_data, innings_stats)

# Print statistics
print("Toss Statistics:")
print(f"Toss Wins: {toss_stats['toss_wins']}")
print(f"Chose to Bat: {toss_stats['bat_first_choice']}")
print(f"Chose to Field: {toss_stats['field_first_choice']}")
print("\nInnings Performance:")
print(innings_stats.to_string(index=False))

# Display visualization
combined_chart

Toss Statistics:
Toss Wins: 7
Chose to Bat: 1
Chose to Field: 6

Innings Performance:
batting_first  matches  wins  losses  win_rate
          Yes        7     6       1      85.7
           No       10     6       4      60.0


alt.HConcatChart(...)

In [34]:
def analyze_match_phases():
    # Add over phase information
    balls_df['phase'] = pd.cut(
        balls_df['over'],
        bins=[-1, 5, 15, 20],
        labels=['Powerplay', 'Middle', 'Death']
    )

    # Calculate phase-wise stats for both batting and bowling
    def get_phase_stats(team_role):
        if team_role == 'batting':
            team_condition = balls_df['batting_team'] == 'Kings XI Punjab'
        else:
            team_condition = balls_df['bowling_team'] == 'Kings XI Punjab'

        phase_stats = balls_df[team_condition].groupby('phase').agg({
            'total_runs': ['sum', 'count'],
            'wicket_type': lambda x: x.notna().sum()
        }).reset_index()

        phase_stats.columns = ['phase', 'runs', 'balls', 'wickets']
        phase_stats['run_rate'] = (phase_stats['runs'] / phase_stats['balls'] * 6).round(2)
        phase_stats['role'] = team_role

        return phase_stats

    batting_stats = get_phase_stats('batting')
    bowling_stats = get_phase_stats('bowling')
    phase_stats = pd.concat([batting_stats, bowling_stats])

    return phase_stats

def create_phase_charts(phase_stats):
    # Run Rate Comparison
    run_rate_chart = alt.Chart(phase_stats).mark_bar().encode(
        x=alt.X('phase:N', title='Match Phase'),
        y=alt.Y('run_rate:Q', title='Run Rate'),
        color=alt.Color('role:N', scale=alt.Scale(
            domain=['batting', 'bowling'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['phase', 'role', 'run_rate', 'runs', 'wickets']
    ).properties(
        width=300,
        height=200,
        title='Run Rate by Phase'
    )

    # Wickets Comparison
    wickets_chart = alt.Chart(phase_stats).mark_bar().encode(
        x=alt.X('phase:N', title='Match Phase'),
        y=alt.Y('wickets:Q', title='Wickets'),
        color=alt.Color('role:N', scale=alt.Scale(
            domain=['batting', 'bowling'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['phase', 'role', 'wickets', 'balls']
    ).properties(
        width=300,
        height=200,
        title='Wickets by Phase'
    )

    combined_chart = alt.hconcat(run_rate_chart, wickets_chart)
    return combined_chart

# Execute analysis
phase_stats = analyze_match_phases()
combined_chart = create_phase_charts(phase_stats)

# Print statistics
print("Phase-wise Performance Summary:")
print("\nBatting:")
print(phase_stats[phase_stats['role'] == 'batting'].to_string(index=False))
print("\nBowling:")
print(phase_stats[phase_stats['role'] == 'bowling'].to_string(index=False))

combined_chart

Phase-wise Performance Summary:

Batting:
    phase  runs  balls  wickets  run_rate    role
Powerplay   866    651       25      7.98 batting
   Middle  1504   1044       43      8.64 batting
    Death   617    358       27     10.34 batting

Bowling:
    phase  runs  balls  wickets  run_rate    role
Powerplay   895    646       26      8.31 bowling
   Middle  1277   1046       50      7.33 bowling
    Death   622    373       37     10.01 bowling


alt.HConcatChart(...)

In [36]:
def analyze_run_rate_progression():
    # Calculate over-by-over stats for batting and bowling
    def calculate_over_stats(team_role):
        # Filter for KXIP batting or bowling
        condition = balls_df['batting_team' if team_role == 'batting' else 'bowling_team'] == 'Kings XI Punjab'

        # Group by over and calculate key metrics
        over_stats = balls_df[condition].groupby('over').agg({
            'total_runs': 'sum',  # Total runs in each over
            'ball': 'count',      # Number of balls in each over
            'wicket_type': lambda x: x.notna().sum()  # Wickets in each over
        }).reset_index()

        over_stats.columns = ['over', 'runs', 'balls', 'wickets']
        over_stats['run_rate'] = (over_stats['runs'] / over_stats['balls'] * 6).round(2)  # Calculate run rate
        over_stats['role'] = team_role
        over_stats['cumulative_runs'] = over_stats['runs'].cumsum()  # Running total of runs

        return over_stats

    # Get stats for both batting and bowling
    batting_stats = calculate_over_stats('batting')
    bowling_stats = calculate_over_stats('bowling')
    over_stats = pd.concat([batting_stats, bowling_stats])

    # Find significant changes in run rate
    for role in ['batting', 'bowling']:
        role_stats = over_stats[over_stats['role'] == role]
        run_rate_change = role_stats['run_rate'].diff()  # Calculate over-to-over change
        # Identify overs with changes larger than one standard deviation
        acceleration_points = role_stats[abs(run_rate_change) > run_rate_change.std()]
        print(f"\n{role.capitalize()} acceleration/deceleration overs:", acceleration_points['over'].tolist())

    return over_stats

def create_progression_charts(over_stats):
    # Base settings for both charts
    width = 400
    height = 200

    # Run Rate Chart
    run_rate_base = alt.Chart(over_stats).encode(
        x=alt.X('over:Q', title='Over', axis=alt.Axis(tickMinStep=1)),
        color=alt.Color('role:N', scale=alt.Scale(
            domain=['batting', 'bowling'],
            range=['#DD1F2D', '#808080']
        ))
    )

    run_rate_lines = run_rate_base.mark_line().encode(
        y=alt.Y('run_rate:Q', title='Runs per Over')
    )

    run_rate_points = run_rate_base.mark_circle().encode(
        y='run_rate:Q',
        tooltip=['over', 'role', 'run_rate', 'runs', 'wickets']
    )

    run_rate_chart = (run_rate_lines + run_rate_points).properties(
        width=width,
        height=height,
        title='Over-by-Over Run Rate'
    )

    # Cumulative Runs Chart
    cumulative_chart = alt.Chart(over_stats).mark_line().encode(
        x=alt.X('over:Q', title='Over', axis=alt.Axis(tickMinStep=1)),
        y=alt.Y('cumulative_runs:Q', title='Total Runs'),
        color=alt.Color('role:N', scale=alt.Scale(
            domain=['batting', 'bowling'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['over', 'role', 'cumulative_runs', 'runs']
    ).properties(
        width=width,
        height=height,
        title='Run Accumulation'
    )

    # Combine charts horizontally
    return alt.hconcat(run_rate_chart, cumulative_chart)

# Execute analysis and display charts
over_stats = analyze_run_rate_progression()
progression_charts = create_progression_charts(over_stats)
progression_charts


Batting acceleration/deceleration overs: [2, 3, 8, 11, 13, 15, 17]

Bowling acceleration/deceleration overs: [3, 6, 10, 13, 14, 19]


alt.HConcatChart(...)

In [39]:
# Calculate boundary stats for batting and bowling
def get_boundary_stats():
    # Create basic boundary counts
    batting_data = balls_df[balls_df['batting_team'] == 'Kings XI Punjab']
    bowling_data = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab']

    # Function to count boundaries
    def count_boundaries(data):
        fours = len(data[data['runs_batsman'] == 4])
        sixes = len(data[data['runs_batsman'] == 6])
        return pd.DataFrame({
            'Boundary': ['Fours', 'Sixes'],
            'Count': [fours, sixes]
        })

    # Get stats by phase
    batting_boundaries = count_boundaries(batting_data)
    batting_boundaries['Role'] = 'Batting'
    bowling_boundaries = count_boundaries(bowling_data)
    bowling_boundaries['Role'] = 'Bowling'

    return pd.concat([batting_boundaries, bowling_boundaries])

# Create visualization
boundary_data = get_boundary_stats()

# Create bar chart
chart = alt.Chart(boundary_data).mark_bar().encode(
    x=alt.X('Role:N', title=None),
    y=alt.Y('Count:Q', title='Number of Boundaries'),
    color=alt.Color('Boundary:N', scale=alt.Scale(
        domain=['Fours', 'Sixes'],
        range=['#DD1F2D', '#808080']
    )),
    tooltip=['Role', 'Boundary', 'Count']
).properties(
    width=400,
    height=300,
    title='KXIP Boundary Analysis'
)

# Display results
print("Boundary Statistics:")
for role in boundary_data['Role'].unique():
    role_data = boundary_data[boundary_data['Role'] == role]
    print(f"\n{role}:")
    print(role_data[['Boundary', 'Count']].to_string(index=False))

chart

Boundary Statistics:

Batting:
Boundary  Count
   Fours    262
   Sixes    127

Bowling:
Boundary  Count
   Fours    240
   Sixes    107


alt.Chart(...)

In [44]:
import pandas as pd
import numpy as np
import altair as alt

def analyze_pressure_moments():
    # Close matches identification
    close_match_summary = pd.DataFrame({
        'Outcome': ['Close Wins', 'Close Losses', 'Other Matches'],
        'Count': [
            len(kxip_matches[
                (((kxip_matches['win_type'] == 'runs') & (kxip_matches['win_margin'] < 20)) |
                ((kxip_matches['win_type'] == 'wickets') & (kxip_matches['win_margin'] <= 3))) &
                (kxip_matches['winner'] == 'Kings XI Punjab')
            ]),
            len(kxip_matches[
                (((kxip_matches['win_type'] == 'runs') & (kxip_matches['win_margin'] < 20)) |
                ((kxip_matches['win_type'] == 'wickets') & (kxip_matches['win_margin'] <= 3))) &
                (kxip_matches['winner'] != 'Kings XI Punjab')
            ]),
            len(kxip_matches[
                ~(((kxip_matches['win_type'] == 'runs') & (kxip_matches['win_margin'] < 20)) |
                ((kxip_matches['win_type'] == 'wickets') & (kxip_matches['win_margin'] <= 3)))
            ])
        ]
    })

    # Death overs analysis
    death_overs = balls_df[balls_df['over'] >= 16].copy()

    # Batting stats
    batting_death = death_overs[death_overs['batting_team'] == 'Kings XI Punjab'].groupby(['match_id', 'over'])['total_runs'].sum().reset_index()
    batting_death['role'] = 'Batting'

    # Bowling stats
    bowling_death = death_overs[death_overs['bowling_team'] == 'Kings XI Punjab'].groupby(['match_id', 'over'])['total_runs'].sum().reset_index()
    bowling_death['role'] = 'Bowling'

    death_stats = pd.concat([batting_death, bowling_death])

    return close_match_summary, death_stats

def create_pressure_charts():
    close_match_summary, death_stats = analyze_pressure_moments()

    # Close matches visualization
    matches_chart = alt.Chart(close_match_summary).mark_bar().encode(
        x=alt.X('Outcome:N', sort=['Close Wins', 'Close Losses', 'Other Matches']),
        y=alt.Y('Count:Q'),
        color=alt.Color('Outcome:N', scale=alt.Scale(
            domain=['Close Wins', 'Close Losses', 'Other Matches'],
            range=['#DD1F2D', '#808080', '#B0B0B0']
        )),
        tooltip=['Outcome', 'Count']
    ).properties(
        width=300,
        height=200,
        title='Match Distribution'
    )

    # Death overs visualization
    death_chart = alt.Chart(death_stats).mark_boxplot().encode(
        x=alt.X('role:N', title='Team Role'),
        y=alt.Y('total_runs:Q', title='Runs per Over'),
        color=alt.Color('role:N', scale=alt.Scale(
            domain=['Batting', 'Bowling'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['role', 'total_runs']
    ).properties(
        width=300,
        height=200,
        title='Death Overs Performance Distribution'
    )

    return alt.hconcat(matches_chart, death_chart)

def analyze_and_display_pressure():
    close_match_summary, death_stats = analyze_pressure_moments()

    # Calculate statistics
    batting_stats = death_stats[death_stats['role'] == 'Batting']
    bowling_stats = death_stats[death_stats['role'] == 'Bowling']

    total_close = close_match_summary[close_match_summary['Outcome'].str.contains('Close')]['Count'].sum()
    close_wins = close_match_summary[close_match_summary['Outcome'] == 'Close Wins']['Count'].iloc[0]

    print(f"""
Pressure Moments Analysis:

Close Matches Summary:
- Total Close Matches: {total_close}
- Close Wins: {close_wins}
- Success Rate: {(close_wins/total_close*100):.1f}%

Death Overs (16-20) Performance:
Batting:
- Average Runs per Over: {batting_stats['total_runs'].mean():.2f}
- Highest Over Score: {batting_stats['total_runs'].max()}
- Lowest Over Score: {batting_stats['total_runs'].min()}

Bowling:
- Average Runs Conceded per Over: {bowling_stats['total_runs'].mean():.2f}
- Highest Runs Conceded: {bowling_stats['total_runs'].max()}
- Lowest Runs Conceded: {bowling_stats['total_runs'].min()}
""")

    return create_pressure_charts()

# Execute analysis
pressure_analysis = analyze_and_display_pressure()
pressure_analysis


Pressure Moments Analysis:

Close Matches Summary:
- Total Close Matches: 2
- Close Wins: 1
- Success Rate: 50.0%

Death Overs (16-20) Performance:
Batting:
- Average Runs per Over: 10.28
- Highest Over Score: 27
- Lowest Over Score: 1

Bowling:
- Average Runs Conceded per Over: 9.87
- Highest Runs Conceded: 26
- Lowest Runs Conceded: 0



alt.HConcatChart(...)

In [46]:
def analyze_partnerships():
    partnerships = []

    for match in kxip_matches['match_id']:
        match_data = balls_df[
            (balls_df['match_id'] == match) &
            (balls_df['batting_team'] == 'Kings XI Punjab')
        ].sort_values(['over', 'ball'])

        current_pair = None
        partnership_runs = 0
        partnership_balls = 0

        for _, ball in match_data.iterrows():
            pair = tuple(sorted([ball['batsman'], ball['non_striker']]))

            if current_pair != pair:
                if current_pair is not None:
                    partnerships.append({
                        'match_id': match,
                        'partnership': ' & '.join(current_pair),
                        'runs': partnership_runs,
                        'balls': partnership_balls
                    })
                current_pair = pair
                partnership_runs = 0
                partnership_balls = 0

            partnership_runs += ball['total_runs']
            partnership_balls += 1

        if current_pair is not None:
            partnerships.append({
                'match_id': match,
                'partnership': ' & '.join(current_pair),
                'runs': partnership_runs,
                'balls': partnership_balls
            })

    partnership_df = pd.DataFrame(partnerships)

    # Calculate partnership statistics
    partnership_stats = partnership_df.groupby('partnership').agg({
        'runs': ['sum', 'count', 'mean'],
        'balls': 'sum'
    }).reset_index()

    partnership_stats.columns = ['partnership', 'total_runs', 'frequency', 'avg_runs', 'total_balls']
    partnership_stats['run_rate'] = (partnership_stats['total_runs'] / partnership_stats['total_balls'] * 6).round(2)
    partnership_stats = partnership_stats.sort_values('total_runs', ascending=False)

    return partnership_stats

def create_partnership_charts(stats):
    # Top partnerships by runs
    runs_chart = alt.Chart(stats.head(10)).mark_bar().encode(
        x=alt.X('partnership:N', sort='-y', title='Partnerships', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('total_runs:Q', title='Total Runs'),
        color=alt.value('#DD1F2D'),
        tooltip=['partnership', 'total_runs', 'frequency', 'run_rate']
    ).properties(
        width=300,
        height=200,
        title='Top 10 Partnerships by Runs'
    )

    # Partnerships by frequency
    freq_data = stats.nlargest(10, 'frequency')
    freq_chart = alt.Chart(freq_data).mark_bar().encode(
        x=alt.X('partnership:N', sort='-y', title='Partnerships', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('frequency:Q', title='Number of Times'),
        color=alt.value('#808080'),
        tooltip=['partnership', 'total_runs', 'frequency', 'run_rate']
    ).properties(
        width=300,
        height=200,
        title='Top 10 Most Frequent Partnerships'
    )

    return alt.hconcat(runs_chart, freq_chart)

# Execute analysis
partnership_stats = analyze_partnerships()

# Print summary statistics
print("Top 5 Partnerships by Runs:")
print(partnership_stats.head()[['partnership', 'total_runs', 'frequency', 'run_rate']].to_string(index=False))
print("\nMost Frequent Partnerships:")
print(partnership_stats.nlargest(5, 'frequency')[['partnership', 'frequency', 'total_runs', 'run_rate']].to_string(index=False))

# Display charts
create_partnership_charts(partnership_stats)

Top 5 Partnerships by Runs:
           partnership  total_runs  frequency  run_rate
DA Miller & GJ Maxwell         385          6     11.11
    M Vohra & V Sehwag         269          8      8.54
     M Vohra & WP Saha         260          3     10.83
 DA Miller & GJ Bailey         196          7     10.23
CA Pujara & GJ Maxwell         158          3      9.39

Most Frequent Partnerships:
           partnership  frequency  total_runs  run_rate
    M Vohra & V Sehwag          8         269      8.54
 DA Miller & GJ Bailey          7         196     10.23
   DA Miller & WP Saha          7         158      8.86
 GJ Maxwell & V Sehwag          7         144      8.64
DA Miller & GJ Maxwell          6         385     11.11


alt.HConcatChart(...)

In [47]:
def analyze_extras():
    # Create function to analyze extras for batting/bowling
    def get_extras_stats(role):
        condition = balls_df['batting_team' if role == 'received' else 'bowling_team'] == 'Kings XI Punjab'
        extras_data = balls_df[condition & balls_df['extras_type'].notna()]

        # Calculate extras by type
        extras_by_type = extras_data.groupby('extras_type').agg({
            'extras_runs': 'sum',
            'match_id': 'count'
        }).reset_index()
        extras_by_type.columns = ['type', 'runs', 'frequency']
        extras_by_type['role'] = role

        # Calculate match impact
        match_extras = extras_data.groupby('match_id')['extras_runs'].sum().reset_index()
        match_result = kxip_matches.set_index('match_id')['winner'].eq('Kings XI Punjab')
        match_extras['result'] = match_extras['match_id'].map(match_result)

        return extras_by_type, match_extras

    # Get stats for both batting and bowling
    batting_extras, batting_impact = get_extras_stats('received')
    bowling_extras, bowling_impact = get_extras_stats('conceded')

    extras_summary = pd.concat([batting_extras, bowling_extras])
    return extras_summary, batting_impact, bowling_impact

def create_extras_charts(extras_summary, batting_impact, bowling_impact):
    # Extras by type chart
    type_chart = alt.Chart(extras_summary).mark_bar().encode(
        x=alt.X('type:N', title='Extra Type'),
        y=alt.Y('runs:Q', title='Runs'),
        color=alt.Color('role:N', scale=alt.Scale(
            domain=['received', 'conceded'],
            range=['#DD1F2D', '#808080']
        )),
        tooltip=['type', 'role', 'runs', 'frequency']
    ).properties(
        width=300,
        height=200,
        title='Extras by Type'
    )

    # Impact visualization
    batting_impact['type'] = 'Received'
    bowling_impact['type'] = 'Conceded'
    impact_data = pd.concat([batting_impact, bowling_impact])

    impact_chart = alt.Chart(impact_data).mark_boxplot().encode(
        x=alt.X('type:N', title='Extra Type'),
        y=alt.Y('extras_runs:Q', title='Extras per Match'),
        color=alt.Color('type:N', scale=alt.Scale(
            domain=['Received', 'Conceded'],
            range=['#DD1F2D', '#808080']
        ))
    ).properties(
        width=300,
        height=200,
        title='Match-wise Extras Distribution'
    )

    return alt.hconcat(type_chart, impact_chart)

# Execute analysis
extras_summary, batting_impact, bowling_impact = analyze_extras()

# Print statistics
print("Extras Analysis Summary:")
for role in ['received', 'conceded']:
    role_data = extras_summary[extras_summary['role'] == role]
    print(f"\nExtras {role.title()}:")
    print(f"Total Runs: {role_data['runs'].sum()}")
    print("By Type:")
    print(role_data[['type', 'runs', 'frequency']].to_string(index=False))

# Display visualization
create_extras_charts(extras_summary, batting_impact, bowling_impact)

Extras Analysis Summary:

Extras Received:
Total Runs: 163
By Type:
   type  runs  frequency
   byes     7          7
legbyes    63         47
noballs    10          9
  wides    83         73

Extras Conceded:
Total Runs: 142
By Type:
   type  runs  frequency
   byes     6          3
legbyes    42         34
noballs    11         11
  wides    83         65


alt.HConcatChart(...)

In [48]:
def analyze_batting_performance():
    # Calculate batting metrics for each player
    batting_data = balls_df[balls_df['batting_team'] == 'Kings XI Punjab']

    def get_player_stats(data):
        stats = data.groupby('batsman').agg({
            'match_id': 'nunique',  # Number of matches
            'runs_batsman': ['sum', 'count'],  # Runs and balls
            'wicket_type': lambda x: x.notna().sum()  # Dismissals
        }).reset_index()

        stats.columns = ['batsman', 'matches', 'runs', 'balls', 'dismissals']

        # Calculate averages and strike rates
        stats['average'] = (stats['runs'] / stats['dismissals']).round(2)
        stats['strike_rate'] = (stats['runs'] / stats['balls'] * 100).round(2)

        # Calculate consistency index (% of innings above 20 runs)
        innings_scores = data.groupby(['match_id', 'batsman'])['runs_batsman'].sum()
        stats['consistency'] = innings_scores.groupby(level=1).apply(
            lambda x: (x >= 20).mean() * 100
        ).round(2)

        return stats.sort_values('runs', ascending=False)

    batting_stats = get_player_stats(batting_data)

    return batting_stats

def create_batting_charts(stats):
    # Top batsmen by runs
    runs_chart = alt.Chart(stats.head(10)).mark_bar().encode(
        x=alt.X('batsman:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('runs:Q'),
        color=alt.value('#DD1F2D'),
        tooltip=['batsman', 'runs', 'average', 'strike_rate']
    ).properties(
        width=300,
        height=200,
        title='Top 10 Run Scorers'
    )

    # Strike rates of top batsmen
    strike_chart = alt.Chart(stats.nlargest(10, 'runs')).mark_bar().encode(
        x=alt.X('batsman:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('strike_rate:Q'),
        color=alt.value('#808080'),
        tooltip=['batsman', 'strike_rate', 'runs', 'average']
    ).properties(
        width=300,
        height=200,
        title='Strike Rates of Top Run Scorers'
    )

    return alt.hconcat(runs_chart, strike_chart)

# Execute analysis
batting_stats = analyze_batting_performance()

# Print statistics
print("Top 5 Batsmen Performance:")
print(batting_stats.head()[['batsman', 'runs', 'average', 'strike_rate', 'consistency']].to_string(index=False))

# Display visualization
create_batting_charts(batting_stats)

Top 5 Batsmen Performance:
   batsman  runs  average  strike_rate  consistency
GJ Maxwell   552    32.47       176.92          NaN
  V Sehwag   455    25.28       136.23          NaN
 DA Miller   446    49.56       145.28          NaN
   WP Saha   362    30.17       141.96          NaN
   M Vohra   324    46.29       132.79          NaN


alt.HConcatChart(...)

In [49]:
def analyze_bowling_performance():
    bowling_data = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab']

    def calculate_bowling_stats(data):
        stats = data.groupby('bowler').agg({
            'match_id': 'nunique',
            'total_runs': 'sum',
            'ball': 'count',
            'wicket_type': lambda x: x.notna().sum()
        }).reset_index()

        stats.columns = ['bowler', 'matches', 'runs', 'balls', 'wickets']
        stats['economy'] = (stats['runs'] / (stats['balls']/6)).round(2)
        stats['strike_rate'] = (stats['balls'] / stats['wickets']).round(2)
        stats['average'] = (stats['runs'] / stats['wickets']).round(2)

        # Phase-wise analysis
        phase_stats = data.groupby(['bowler', 'phase']).agg({
            'total_runs': 'sum',
            'ball': 'count',
            'wicket_type': lambda x: x.notna().sum()
        }).reset_index()

        return stats.sort_values('wickets', ascending=False), phase_stats

    bowling_stats, phase_stats = calculate_bowling_stats(bowling_data)
    return bowling_stats, phase_stats

def create_bowling_charts(stats, phase_stats):
    # Wickets and economy rate chart
    performance_chart = alt.Chart(stats.head(8)).mark_bar().encode(
        x=alt.X('bowler:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('wickets:Q', title='Wickets'),
        color=alt.value('#DD1F2D'),
        tooltip=['bowler', 'wickets', 'economy', 'average']
    ).properties(
        width=300,
        height=200,
        title='Top Wicket Takers'
    )

    # Phase-wise wickets
    phase_chart = alt.Chart(
        phase_stats[phase_stats['bowler'].isin(stats.head(8)['bowler'])]
    ).mark_bar().encode(
        x=alt.X('bowler:N', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('wicket_type:Q', title='Wickets'),
        color=alt.Color('phase:N', scale=alt.Scale(
            domain=['Powerplay', 'Middle', 'Death'],
            range=['#DD1F2D', '#808080', '#B0B0B0']
        )),
        tooltip=['bowler', 'phase', 'wicket_type', 'total_runs']
    ).properties(
        width=300,
        height=200,
        title='Phase-wise Wickets'
    )

    return alt.hconcat(performance_chart, phase_chart)

# Execute analysis
bowling_stats, phase_stats = analyze_bowling_performance()

print("Top Bowlers Performance:")
print(bowling_stats.head()[['bowler', 'wickets', 'economy', 'strike_rate', 'average']].to_string(index=False))

# Display charts
create_bowling_charts(bowling_stats, phase_stats)

Top Bowlers Performance:
         bowler  wickets  economy  strike_rate  average
       AR Patel       19     6.31        21.21    22.32
 Sandeep Sharma       19     8.42        13.42    18.84
     MG Johnson       17     7.94        20.00    26.47
       R Dhawan       15     7.61        19.60    24.87
Karanveer Singh       12     7.81        10.50    13.67


alt.HConcatChart(...)

In [50]:
def analyze_matchups():
    # Get batsmen facing KXIP bowlers
    bowling_matchups = balls_df[balls_df['bowling_team'] == 'Kings XI Punjab'].groupby(['bowler', 'batsman']).agg({
        'total_runs': ['sum', 'count'],
        'wicket_type': lambda x: x.notna().sum()
    }).reset_index()

    bowling_matchups.columns = ['bowler', 'batsman', 'runs', 'balls', 'wickets']
    bowling_matchups = bowling_matchups[bowling_matchups['balls'] >= 6]  # Min 1 over
    bowling_matchups['economy'] = (bowling_matchups['runs'] / (bowling_matchups['balls']/6)).round(2)

    # Get KXIP batsmen against bowlers
    batting_matchups = balls_df[balls_df['batting_team'] == 'Kings XI Punjab'].groupby(['batsman', 'bowler']).agg({
        'runs_batsman': ['sum', 'count'],
        'wicket_type': lambda x: x.notna().sum()
    }).reset_index()

    batting_matchups.columns = ['batsman', 'bowler', 'runs', 'balls', 'wickets']
    batting_matchups = batting_matchups[batting_matchups['balls'] >= 6]
    batting_matchups['strike_rate'] = (batting_matchups['runs'] / batting_matchups['balls'] * 100).round(2)

    return bowling_matchups, batting_matchups

def create_matchup_charts(bowling_matchups, batting_matchups):
    # Best bowling matchups
    bowling_chart = alt.Chart(
        bowling_matchups.nsmallest(10, 'economy')
    ).mark_bar().encode(
        x=alt.X('economy:Q', title='Economy Rate'),
        y=alt.Y('bowler:N', sort='x'),
        color=alt.value('#DD1F2D'),
        tooltip=['bowler', 'batsman', 'economy', 'wickets', 'runs', 'balls']
    ).properties(
        width=300,
        height=200,
        title='Best Bowling Matchups'
    )

    # Best batting matchups
    batting_chart = alt.Chart(
        batting_matchups.nlargest(10, 'strike_rate')
    ).mark_bar().encode(
        x=alt.X('strike_rate:Q', title='Strike Rate'),
        y=alt.Y('batsman:N', sort='-x'),
        color=alt.value('#808080'),
        tooltip=['batsman', 'bowler', 'strike_rate', 'runs', 'balls']
    ).properties(
        width=300,
        height=200,
        title='Best Batting Matchups'
    )

    return alt.hconcat(bowling_chart, batting_chart)

# Execute analysis
bowling_matchups, batting_matchups = analyze_matchups()

# Print stats
print("Most Favorable Bowling Matchups:")
print(bowling_matchups.nsmallest(5, 'economy')[['bowler', 'batsman', 'economy', 'wickets']].to_string(index=False))
print("\nMost Favorable Batting Matchups:")
print(batting_matchups.nlargest(5, 'strike_rate')[['batsman', 'bowler', 'strike_rate', 'runs']].to_string(index=False))

# Display charts
create_matchup_charts(bowling_matchups, batting_matchups)

Most Favorable Bowling Matchups:
         bowler      batsman  economy  wickets
  Shivam Sharma Yuvraj Singh     0.86        1
  Shivam Sharma     AJ Finch     1.71        1
Karanveer Singh  JDS Neesham     2.25        1
       R Dhawan Yuvraj Singh     2.40        1
       AR Patel    YK Pathan     2.73        0

Most Favorable Batting Matchups:
   batsman       bowler  strike_rate  runs
 GJ Bailey    MM Sharma       340.00    34
 DA Miller     HV Patel       316.67    19
   M Vohra MC Henriques       300.00    18
   WP Saha MC Henriques       300.00    18
GJ Maxwell Yuvraj Singh       283.33    17


alt.HConcatChart(...)

In [51]:
def analyze_field_placements():
    # Filter dismissals with fielder involvement
    fielding_data = balls_df[
        (balls_df['bowling_team'] == 'Kings XI Punjab') &
        (balls_df['wicket_fielders'].notna())
    ]

    # Extract fielding positions when available
    def get_fielding_positions():
        caught_data = fielding_data[fielding_data['wicket_type'].str.contains('caught|stumped', na=False)]
        return caught_data

    # Analyze run saving patterns
    def analyze_run_prevention():
        non_boundary_data = balls_df[
            (balls_df['bowling_team'] == 'Kings XI Punjab') &
            (balls_df['runs_batsman'] < 4)
        ]
        return non_boundary_data.groupby('over')['runs_batsman'].mean().reset_index()

    caught_data = get_fielding_positions()
    run_prevention = analyze_run_prevention()

    return caught_data, run_prevention

def create_fielding_charts(caught_data, run_prevention):
    # Catching positions effectiveness
    catches_chart = alt.Chart(caught_data).mark_bar().encode(
        x=alt.X('wicket_type:N', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('count():Q', title='Number of Dismissals'),
        color=alt.value('#DD1F2D'),
        tooltip=['wicket_type', 'count()']
    ).properties(
        width=300,
        height=200,
        title='Dismissal Types'
    )

    # Run prevention pattern
    prevention_chart = alt.Chart(run_prevention).mark_line().encode(
        x=alt.X('over:Q', title='Over'),
        y=alt.Y('runs_batsman:Q', title='Average Runs Conceded'),
        color=alt.value('#808080'),
        tooltip=['over', 'runs_batsman']
    ).properties(
        width=300,
        height=200,
        title='Run Prevention Pattern'
    )

    return alt.hconcat(catches_chart, prevention_chart)

# Execute analysis
caught_data, run_prevention = analyze_field_placements()

print("Fielding Analysis:")
print("\nDismissal Distribution:")
print(caught_data['wicket_type'].value_counts().head().to_string())
print("\nRun Prevention by Phase:")
phase_prevention = run_prevention.copy()
phase_prevention['phase'] = pd.cut(phase_prevention['over'],
                                 bins=[-1, 5, 15, 20],
                                 labels=['Powerplay', 'Middle', 'Death'])
print(phase_prevention.groupby('phase')['runs_batsman'].mean().round(2).to_string())

# Display charts
create_fielding_charts(caught_data, run_prevention)

Fielding Analysis:

Dismissal Distribution:
wicket_type
caught     69
stumped     4

Run Prevention by Phase:
phase
Powerplay    0.45
Middle       0.67
Death        0.70


alt.HConcatChart(...)

In [52]:
def analyze_death_specialists():
    death_overs = balls_df[balls_df['over'] >= 16]

    # Batting analysis in death overs
    batting_death = death_overs[death_overs['batting_team'] == 'Kings XI Punjab']
    batting_stats = batting_death.groupby('batsman').agg({
        'runs_batsman': ['sum', 'count'],
        'match_id': 'nunique',
        'wicket_type': lambda x: x.notna().sum()
    }).reset_index()

    batting_stats.columns = ['player', 'runs', 'balls', 'matches', 'dismissals']
    batting_stats['strike_rate'] = (batting_stats['runs'] / batting_stats['balls'] * 100).round(2)
    batting_stats['role'] = 'Batting'

    # Bowling analysis in death overs
    bowling_death = death_overs[death_overs['bowling_team'] == 'Kings XI Punjab']
    bowling_stats = bowling_death.groupby('bowler').agg({
        'total_runs': 'sum',
        'ball': 'count',
        'match_id': 'nunique',
        'wicket_type': lambda x: x.notna().sum()
    }).reset_index()

    bowling_stats.columns = ['player', 'runs', 'balls', 'matches', 'wickets']
    bowling_stats['economy'] = (bowling_stats['runs'] / (bowling_stats['balls']/6)).round(2)
    bowling_stats['role'] = 'Bowling'

    return batting_stats, bowling_stats

def create_death_specialists_charts(batting_stats, bowling_stats):
    # Best death batsmen
    batting_chart = alt.Chart(
        batting_stats[batting_stats['balls'] >= 30].nlargest(8, 'strike_rate')
    ).mark_bar().encode(
        x=alt.X('player:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('strike_rate:Q', title='Strike Rate'),
        color=alt.value('#DD1F2D'),
        tooltip=['player', 'strike_rate', 'runs', 'balls']
    ).properties(
        width=300,
        height=200,
        title='Best Death Overs Batsmen'
    )

    # Best death bowlers
    bowling_chart = alt.Chart(
        bowling_stats[bowling_stats['balls'] >= 30].nsmallest(8, 'economy')
    ).mark_bar().encode(
        x=alt.X('player:N', sort='y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('economy:Q', title='Economy Rate'),
        color=alt.value('#808080'),
        tooltip=['player', 'economy', 'wickets', 'balls']
    ).properties(
        width=300,
        height=200,
        title='Best Death Overs Bowlers'
    )

    return alt.hconcat(batting_chart, bowling_chart)

batting_stats, bowling_stats = analyze_death_specialists()

print("Death Overs (16-20) Specialists:")
print("\nTop 5 Batsmen:")
print(batting_stats[batting_stats['balls'] >= 30].nlargest(5, 'strike_rate')[
    ['player', 'strike_rate', 'runs', 'balls']
].to_string(index=False))

print("\nTop 5 Bowlers:")
print(bowling_stats[bowling_stats['balls'] >= 30].nsmallest(5, 'economy')[
    ['player', 'economy', 'wickets', 'runs']
].to_string(index=False))

create_death_specialists_charts(batting_stats, bowling_stats)

Death Overs (16-20) Specialists:

Top 5 Batsmen:
    player  strike_rate  runs  balls
 DA Miller       206.35   130     63
   WP Saha       205.13    80     39
 GJ Bailey       188.00   188    100
  R Dhawan       135.00    54     40
MG Johnson       109.80    56     51

Top 5 Bowlers:
      player  economy  wickets  runs
    AR Patel     7.42        5    68
  MG Johnson     8.07        9   109
    R Dhawan    10.20        3    68
BE Hendricks    10.45        4    54
    L Balaji    11.71        4   123


alt.HConcatChart(...)

In [53]:
def analyze_powerplay():
    powerplay = balls_df[balls_df['over'] < 6]

    # Batting analysis
    batting_pp = powerplay[powerplay['batting_team'] == 'Kings XI Punjab']
    batting_stats = batting_pp.groupby('batsman').agg({
        'runs_batsman': ['sum', 'count'],
        'match_id': 'nunique',
        'wicket_type': lambda x: x.notna().sum()
    }).reset_index()
    batting_stats.columns = ['player', 'runs', 'balls', 'matches', 'dismissals']
    batting_stats['strike_rate'] = (batting_stats['runs'] / batting_stats['balls'] * 100).round(2)

    # Bowling analysis
    bowling_pp = powerplay[powerplay['bowling_team'] == 'Kings XI Punjab']
    bowling_stats = bowling_pp.groupby('bowler').agg({
        'total_runs': 'sum',
        'ball': 'count',
        'wicket_type': lambda x: x.notna().sum(),
        'match_id': 'nunique'
    }).reset_index()
    bowling_stats.columns = ['player', 'runs', 'balls', 'wickets', 'matches']
    bowling_stats['economy'] = (bowling_stats['runs'] / (bowling_stats['balls']/6)).round(2)

    return batting_stats, bowling_stats

def create_powerplay_charts(batting_stats, bowling_stats):
    batting_chart = alt.Chart(
        batting_stats[batting_stats['balls'] >= 20].nlargest(8, 'strike_rate')
    ).mark_bar().encode(
        x=alt.X('player:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('strike_rate:Q', title='Strike Rate'),
        color=alt.value('#DD1F2D'),
        tooltip=['player', 'strike_rate', 'runs', 'balls', 'matches']
    ).properties(
        width=300,
        height=200,
        title='Top Powerplay Batsmen'
    )

    bowling_chart = alt.Chart(
        bowling_stats[bowling_stats['balls'] >= 20].nsmallest(8, 'economy')
    ).mark_bar().encode(
        x=alt.X('player:N', sort='y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('economy:Q', title='Economy Rate'),
        color=alt.value('#808080'),
        tooltip=['player', 'economy', 'wickets', 'balls', 'matches']
    ).properties(
        width=300,
        height=200,
        title='Best Powerplay Bowlers'
    )

    return alt.hconcat(batting_chart, bowling_chart)

# Execute analysis
batting_stats, bowling_stats = analyze_powerplay()

print("Powerplay Performers (First 6 overs):")
print("\nTop 5 Batsmen:")
print(batting_stats[batting_stats['balls'] >= 20].nlargest(5, 'strike_rate')[
    ['player', 'strike_rate', 'runs', 'matches']
].to_string(index=False))

print("\nTop 5 Bowlers:")
print(bowling_stats[bowling_stats['balls'] >= 20].nsmallest(5, 'economy')[
    ['player', 'economy', 'wickets', 'matches']
].to_string(index=False))

create_powerplay_charts(batting_stats, bowling_stats)

Powerplay Performers (First 6 overs):

Top 5 Batsmen:
    player  strike_rate  runs  matches
GJ Maxwell       150.00    69        8
   M Vohra       145.83   210        8
   WP Saha       130.12   108        8
  V Sehwag       126.13   280       17
 CA Pujara        88.89    80        6

Top 5 Bowlers:
        player  economy  wickets  matches
      AR Patel     6.96        0        8
      L Balaji     7.06        4        8
Sandeep Sharma     7.50       13       11
    MG Johnson     8.01        6       14
       P Awana    10.02        3        6


alt.HConcatChart(...)

In [54]:
def analyze_scoring_patterns():
    # Filter KXIP batting data
    batting_data = balls_df[balls_df['batting_team'] == 'Kings XI Punjab']

    # Classify runs into categories
    batting_data['run_type'] = pd.cut(
        batting_data['runs_batsman'],
        bins=[-1, 0, 1, 2, 3, 4, 6],
        labels=['Dot', 'Singles', 'Doubles', 'Threes', 'Fours', 'Sixes']
    )

    # Player-wise analysis
    player_stats = batting_data.groupby('batsman').agg({
        'runs_batsman': 'sum',
        'ball': 'count'
    }).reset_index()

    # Calculate running stats
    running_stats = batting_data[batting_data['runs_batsman'].between(1, 3)].groupby('batsman').agg({
        'ball': 'count',
        'runs_batsman': 'sum'
    }).reset_index()
    running_stats.columns = ['batsman', 'running_balls', 'running_runs']

    # Calculate boundary stats
    boundary_stats = batting_data[batting_data['runs_batsman'].isin([4, 6])].groupby('batsman').agg({
        'ball': 'count',
        'runs_batsman': 'sum'
    }).reset_index()
    boundary_stats.columns = ['batsman', 'boundary_balls', 'boundary_runs']

    # Combine stats
    combined_stats = player_stats.merge(running_stats, on='batsman', how='left')
    combined_stats = combined_stats.merge(boundary_stats, on='batsman', how='left')
    combined_stats = combined_stats.fillna(0)

    # Calculate percentages
    combined_stats['running_percentage'] = (combined_stats['running_runs'] / combined_stats['runs_batsman'] * 100).round(2)
    combined_stats['boundary_percentage'] = (combined_stats['boundary_runs'] / combined_stats['runs_batsman'] * 100).round(2)

    return combined_stats, batting_data['run_type'].value_counts()

def create_scoring_charts(combined_stats, run_distribution):
    # Top players chart
    players_chart = alt.Chart(
        combined_stats[combined_stats['runs_batsman'] >= 100].nlargest(8, 'runs_batsman')
    ).mark_bar().encode(
        x=alt.X('batsman:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('runs_batsman:Q'),
        color=alt.value('#DD1F2D'),
        tooltip=['batsman', 'runs_batsman', 'running_percentage', 'boundary_percentage']
    ).properties(
        width=300,
        height=200,
        title='Top Batsmen Run Distribution'
    )

    # Run type distribution
    distribution_data = pd.DataFrame({
        'type': run_distribution.index,
        'count': run_distribution.values
    })

    distribution_chart = alt.Chart(distribution_data).mark_bar().encode(
        x=alt.X('type:N', sort=['Dot', 'Singles', 'Doubles', 'Threes', 'Fours', 'Sixes']),
        y=alt.Y('count:Q'),
        color=alt.value('#808080'),
        tooltip=['type', 'count']
    ).properties(
        width=300,
        height=200,
        title='Ball-by-Ball Run Distribution'
    )

    return alt.hconcat(players_chart, distribution_chart)

# Execute analysis
combined_stats, run_distribution = analyze_scoring_patterns()

# Print key statistics
print("Scoring Pattern Analysis (min 100 runs):")
print(combined_stats[combined_stats['runs_batsman'] >= 100].nlargest(5, 'runs_batsman')[
    ['batsman', 'runs_batsman', 'running_percentage', 'boundary_percentage']
].to_string(index=False))

create_scoring_charts(combined_stats, run_distribution)

Scoring Pattern Analysis (min 100 runs):
   batsman  runs_batsman  running_percentage  boundary_percentage
GJ Maxwell           552               26.09                73.91
  V Sehwag           455               27.03                72.97
 DA Miller           446               41.26                58.74
   WP Saha           362               33.70                66.30
   M Vohra           324               35.49                62.96


alt.HConcatChart(...)

In [55]:
def analyze_situational_performance():
    # Identify chasing/setting situations
    def get_match_situation():
        situation_data = pd.merge(balls_df, kxip_matches[['match_id', 'toss_winner', 'toss_decision', 'winner']], on='match_id')
        situation_data['is_chasing'] = (
            ((situation_data['batting_team'] == 'Kings XI Punjab') &
             (situation_data['toss_winner'] == 'Kings XI Punjab') &
             (situation_data['toss_decision'] == 'field')) |
            ((situation_data['batting_team'] == 'Kings XI Punjab') &
             (situation_data['toss_winner'] != 'Kings XI Punjab') &
             (situation_data['toss_decision'] == 'bat'))
        )
        return situation_data

    situation_data = get_match_situation()

    # Player performance in different situations
    def get_player_situations(data):
        batting_stats = data[data['batting_team'] == 'Kings XI Punjab'].groupby(
            ['batsman', 'is_chasing']
        ).agg({
            'runs_batsman': ['sum', 'count'],
            'match_id': 'nunique'
        }).reset_index()

        batting_stats.columns = ['player', 'is_chasing', 'runs', 'balls', 'matches']
        batting_stats['strike_rate'] = (batting_stats['runs'] / batting_stats['balls'] * 100).round(2)
        batting_stats['role'] = 'Batting'

        return batting_stats

    situation_stats = get_player_situations(situation_data)

    return situation_stats

def create_situation_charts(situation_stats):
    # Batting performance comparison
    chase_chart = alt.Chart(
        situation_stats[situation_stats['runs'] >= 100]
    ).mark_bar().encode(
        x=alt.X('player:N', sort='-y', axis=alt.Axis(labelAngle=45)),
        y=alt.Y('strike_rate:Q', title='Strike Rate'),
        color=alt.Color('is_chasing:N',
            scale=alt.Scale(domain=[True, False], range=['#DD1F2D', '#808080']),
            legend=alt.Legend(title='Chasing?')),
        tooltip=['player', 'strike_rate', 'runs', 'matches']
    ).properties(
        width=600,
        height=300,
        title='Player Performance: Chasing vs Setting Targets'
    )

    return chase_chart

# Execute analysis
situation_stats = analyze_situational_performance()

# Print summary stats
print("\nTop Performers in Chases (min 100 runs):")
print(situation_stats[
    (situation_stats['is_chasing'] == True) &
    (situation_stats['runs'] >= 100)
].nlargest(5, 'strike_rate')[
    ['player', 'runs', 'strike_rate', 'matches']
].to_string(index=False))

print("\nTop Performers Setting Targets (min 100 runs):")
print(situation_stats[
    (situation_stats['is_chasing'] == False) &
    (situation_stats['runs'] >= 100)
].nlargest(5, 'strike_rate')[
    ['player', 'runs', 'strike_rate', 'matches']
].to_string(index=False))

create_situation_charts(situation_stats)


Top Performers in Chases (min 100 runs):
    player  runs  strike_rate  matches
GJ Maxwell   253       170.95        7
   M Vohra   162       157.28        4
 GJ Bailey   101       144.29        6
 DA Miller   212       134.18        7
   WP Saha   106       127.71        5

Top Performers Setting Targets (min 100 runs):
    player  runs  strike_rate  matches
GJ Maxwell   299       182.32        9
 DA Miller   234       157.05        9
   WP Saha   256       148.84        8
  V Sehwag   364       143.87       10
 GJ Bailey   156       130.00       10


alt.Chart(...)